<h2>Project #2 - Web Scraping Billboard And Spotify API</h2>
Sergey Khegay

### Import necessary Libraries

In [1]:
# importing the libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
from tqdm.notebook import tqdm, trange
from datetime import datetime
import time    # to be used in loop iterations

### Define Functions and Lists for Web Scarping

Define function to web scrape **artist name's** from given Billboard link and store it in **Artists** list.

In [2]:
Artists = []
def webscraping_name(link_url,list_name):
    url_content = requests.get(link_url)
    soup = BeautifulSoup(url_content.content, 'html.parser')
    name_list = soup.find_all('span', class_="chart-element__information__artist")
    for i in name_list:
        list_name.append(i.get_text())
    print('Artist name\'s  web scraped.')

Define function to web scrape **name of the songs**  from given Billboard link and store it in **Songs** list.

In [3]:
Songs = []
Peak = []
def webscraping_songs(link_url,list_song,list_peak):
    url_content = requests.get(link_url)
    soup = BeautifulSoup(url_content.content, 'html.parser')
    name_list = soup.find_all('span', class_="chart-element__information__song")
    for i in name_list:
        list_song.append(i.get_text())
    peak_list = soup.find_all('span', class_="chart-element__meta text--center color--secondary text--peak")
    for i in peak_list:
        list_peak.append(i.get_text())
    print('Billboard track\'s web scraped.')

### Run Web Scraping Functions and Create Dataframe

Define **Billboard URL** and run the funtions.

In [4]:
url = "https://www.billboard.com/charts/billboard-global-200"
webscraping_name(url,Artists)
webscraping_songs(url,Songs,Peak)

Artist name's  web scraped.
Billboard track's web scraped.


Create **Billboard dataframe** and assign lists to relative columns.

In [5]:
df_billboard = pd.DataFrame()
df_billboard['Song'] = Songs
df_billboard['Artist'] = Artists
df_billboard['Billboard Peak'] = Peak
#df_billboard

### Define Spotify Credentials and Access

In [6]:
client_id = '2ebbbedc6bd84d989c84ca1222925180'#{spotify client id}
client_secret = '8740693f8ff8484b9218408e79620cca'#{spotify secret id}
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

### Define Functions and Lists to Return Information Using API

Define function to get **track uri** from Spotify request and store data in the **track_uris** list.

In [7]:
#Track uri List
track_uris = []

In [8]:
def get_track_uris(song_list,artist_list,uri_list):
    with tqdm(total=len(song_list)) as p_bar:
        for i in range(0, len(song_list)):
            p_bar.set_description(f'Working on "{i+1}"')
            p_bar.update(1)
            song = str(artist_list[i])+','+str(song_list[i])
            search = sp.search(song)
            if search['tracks']['total']==0:
                song = str(song_list[i])
                search = sp.search(song)
            track_uri = search['tracks']['items'][0]['uri']
            uri_list.append(track_uri)

Define function to request **audio features** and store it in relative lists using **track uri**.

In [9]:
#Audio Features List
acousticness = []
danceability = []
energy = []
instrumentalness = []
liveness = []
loudness = []
speechiness = []
valence = []
tempo = []
key = []
mode = []
time_sig = []
duration = []

In [10]:
def audio_features(uri_list):
    with tqdm(total=len(uri_list)) as p_bar:
        count = 1
        for i in uri_list:
            p_bar.set_description(f'Working on "{count}"')
            count +=1
            p_bar.update(1)
            features = sp.audio_features(i)
            acousticness.append(features[0]['acousticness'])
            danceability.append(features[0]['danceability'])
            energy.append(features[0]['energy'])
            instrumentalness.append(features[0]['instrumentalness'])
            liveness.append(features[0]['liveness'])
            loudness.append(features[0]['loudness'])
            speechiness.append(features[0]['speechiness'])
            tempo.append(features[0]['tempo'])
            valence.append(features[0]['valence'])
            key.append(features[0]['key'])
            mode.append(features[0]['mode'])
            time_sig.append(features[0]['time_signature'])
            duration.append(features[0]['duration_ms'])

Define function to request additional information including **album, track popularity, release data**, and whether the track is **explicit**. 

In [11]:
#Track Info List
popularity = []
album_name = []
popularity = []
explicit = []
release_date = []

In [12]:
#Additional Track Information
def track_information(uri_list):
    with tqdm(total=len(uri_list)) as p_bar:
        count = 1
        for i in uri_list:
            p_bar.set_description(f'Working on "{count}"')
            p_bar.update(1)
            count +=1
            track_info = sp.track(i)
            popularity.append(track_info['popularity'])
            album_name.append(track_info['album']['name'])
            explicit.append(track_info['explicit'])
            release_date.append(track_info['album']['release_date'])

### Run API Functions Get Track Infromation From Spotify and Create Dataframe
Request all information using Spotify API request.

In [13]:
get_track_uris(Songs,Artists,track_uris)
track_information(track_uris)
audio_features(track_uris) 

Create **Spotify dataframe** and assign lists to relative columns.

In [14]:
df_spotify = pd.DataFrame()
df_spotify['Track_URI'] = track_uris
df_spotify['Explicit'] = explicit
df_spotify['Album Name'] = album_name
df_spotify['Release Date'] = release_date
df_spotify['Popularity'] = popularity
df_spotify['Acousticness'] = acousticness
df_spotify['Danceability'] = danceability
df_spotify['Energy'] = energy
df_spotify['Instrumentalness'] = instrumentalness
df_spotify['Liveness'] = liveness
df_spotify['Loudness'] = loudness
df_spotify['Speechiness'] = speechiness
df_spotify['Valence'] = valence
df_spotify['Tempo'] = tempo
df_spotify['Key'] = key
df_spotify['Mode'] = mode
df_spotify['Time Signature'] = time_sig
df_spotify['Duration(ms)'] = duration

### Data Transformation

Converting milliseconds to minutes.

In [15]:
def convert(ms):
    time = datetime.fromtimestamp(ms/1000).strftime('%M:%S')
    return time
df_spotify['Duration(ms)']=df_spotify['Duration(ms)'].apply(convert)

Converting integer key value to associated tonal counterpart.

In [16]:
def key(key):
    if key==0:
        return 'C'
    elif key==1:
        return 'C♯/D♭'
    elif key==2:
        return 'D'
    elif key==3:
        return 'D♯/E♭'
    elif key==4:
        return 'E'
    elif key==5:
        return 'F'
    elif key==6:
        return 'F♯/G♭'
    elif key==7:
        return 'G'
    elif key==8:
        return 'G♯/A♭'
    elif key==9:
        return 'A'
    elif key==9:
        return 'A♯/B♭'
    elif key==10:
        return 'B'
    else:
        return 'N/A'
df_spotify['Key'] = df_spotify['Key'].apply(key)

Converting Mode to Major or Minor.

In [17]:
def mode(mode):
    if mode==0:
        return 'Minor'
    elif mode==1:
        return 'Major'
df_spotify['Mode'] = df_spotify['Mode'].apply(mode)

### Combine Dataframes, Produce Descriptive Statisic ,and Export Dataframe to CSV

Combine **Billboard** and **Spotify** Dataframes.

In [18]:
final_df = pd.concat([df_billboard, df_spotify], axis=1, sort=False)
final_df

,Song,Artist,Billboard Peak,Track_URI,Explicit,Album Name,Release Date,Popularity,Acousticness,Danceability,...,Instrumentalness,Liveness,Loudness,Speechiness,Valence,Tempo,Key,Mode,Time Signature,Duration(ms)
0,Montero (Call Me By Your Name),Lil Nas X,1,spotify:track:67BtfxlNbhBmCDR2L2l8qd,True,MONTERO (Call Me By Your Name),2021-03-31,99,0.29700,0.610,...,0.000000,0.3840,-6.682,0.1520,0.758,178.818,G♯/A♭,Minor,4,02:17
1,Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,1,spotify:track:4iJyoBOLtHqaGxP12qzhQI,True,Justice,2021-03-19,100,0.32100,0.677,...,0.000000,0.4200,-6.181,0.1190,0.464,90.030,C,Major,4,03:18
2,Levitating,Dua Lipa Featuring DaBaby,3,spotify:track:463CkQjx2Zk1yXoBuierM9,False,Levitating (feat. DaBaby),2020-10-01,91,0.00883,0.702,...,0.000000,0.0674,-3.787,0.0601,0.915,102.977,F♯/G♭,Minor,4,03:23
3,Save Your Tears,The Weeknd,1,spotify:track:37BZB0z9T8Xu7U3e65qxFy,False,Save Your Tears (Remix),2021-04-23,94,0.02150,0.650,...,0.000024,0.0936,-4.645,0.0325,0.593,118.091,C,Major,4,03:11
4,Kiss Me More,Doja Cat Featuring SZA,5,spotify:track:748mdHapucXQri7IAO8yFK,True,Kiss Me More (feat. SZA),2021-04-09,98,0.23500,0.762,...,0.000158,0.1230,-3.541,0.0286,0.742,110.968,G♯/A♭,Major,4,03:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Somebody That I Used To Know,Gotye Featuring Kimbra,141,spotify:track:4wCmqSrbyCgxEXROQE6vtV,False,Making Mirrors,2011-01-01,77,0.59100,0.864,...,0.000133,0.0992,-7.036,0.0370,0.720,129.062,C,Major,4,04:04
196,Every Breath You Take,The Police,197,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,False,Synchronicity (Remastered 2003),1983-06-17,84,0.54300,0.820,...,0.002940,0.0714,-9.796,0.0348,0.740,117.401,C♯/D♭,Major,4,04:13
197,Lonely,Justin Bieber & benny blanco,5,spotify:track:4fKiuIavbsoTRZ6R7xuAyT,True,Lonely (with benny blanco) [Main + Acoustic],2020-10-27,59,0.96900,0.365,...,0.000002,0.6840,-10.610,0.0410,0.119,168.744,N/A,Minor,4,02:35
198,Tennessee Whiskey,Chris Stapleton,179,spotify:track:3fqwjXwUGN6vbzIwvyFMhx,False,Traveller,2015-05-04,79,0.20500,0.392,...,0.009600,0.0821,-10.888,0.0298,0.512,48.718,A,Major,4,04:53


Using **.describe()** function produce descriptive Statistics.

In [19]:
final_df.describe()

,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Valence,Tempo,Time Signature
count,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,76.545000,0.256809,0.67276,0.606000,0.058808,0.168785,-6.783775,0.113762,0.507020,121.858770,3.980000
std,23.325057,0.268565,0.15347,0.179819,0.201666,0.128056,2.997011,0.108443,0.227467,28.714145,0.223269
min,0.000000,0.000025,0.23600,0.117000,0.000000,0.034400,-17.573000,0.023200,0.060500,48.718000,3.000000
25%,78.750000,0.036800,0.57800,0.490250,0.000000,0.095100,-8.085250,0.040375,0.333250,98.728250,4.000000
50%,84.000000,0.174500,0.68750,0.608500,0.000000,0.119500,-6.303500,0.060100,0.509000,121.478500,4.000000
75%,88.000000,0.401750,0.79425,0.729250,0.000139,0.185500,-4.662250,0.135250,0.680500,139.995000,4.000000
max,100.000000,0.987000,0.96500,0.973000,0.962000,0.790000,-1.412000,0.619000,0.962000,199.844000,5.000000


Store Results as **billboard.csv** file.

In [20]:
final_df.to_csv('billboard.csv')